In [1]:
import pandas as pd
from datetime import datetime, timedelta
from geopy.distance import geodesic

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [12]:
shike = pd.read_excel('C:/intern/data/docs_20220525/时刻表/shike.xlsx')

In [69]:
def append_data(files):
    appended_data = pd.DataFrame(columns=['日期','站点','距离','车次'])
    for f in files:
        each_data = pd.read_excel(f)
        appended_data = appended_data.append(each_data)
    return appended_data

In [70]:
def check_different_zhandian(before_df, after_df, riqi, checi):
    before_zhandian_set = set(before_df[(before_df['日期']==riqi) & (before_df['车次']==checi)]['站点'].values)
    after_zhandian_set = set(after_df[(after_df['日期']==riqi) & (after_df['车次']==checi)]['站点'].values)
    diff_chandian = after_zhandian_set.difference(before_zhandian_set)
    
    return diff_chandian

In [78]:
def checi_duiying_luxian(checi):
    return shike[shike['车次id']==checi]['路线'].values[0]

In [85]:
def check_zhandian_included_checi(zhandians, checi):
    zhandian_list = list(zhandians)
    res = []
    for zhandian_ in zhandian_list:
        val = shike[(shike['站点']==zhandian_)&(shike['车次id']==checi)]
        if len(val)>0:
            res.append(zhandian_)
            
    return res

In [91]:
def check_xinzeng_valid(xinzeng):
    if len(xinzeng)>0:
        return True
    else:
        return False

In [126]:
def check_zhandian_distance(df, zhandians, riqi, checi):
    dist_list = []
    for zhandian_ in zhandians:
        dist_ = df[(df['日期']==riqi) & (df['车次']==checi) & (df['站点']==zhandian_)]['距离'].values[0]
        dist_list.append(dist_)
    return dist_list

In [133]:
file = ['C:/intern/data/docs_20220525/插值前后经停靠站点数对比0-90.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比90-149.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比150-221.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比244-299.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比300-399.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比400-499.xlsx',
        'C:/intern/data/docs_20220525/插值前后经停靠站点数对比500-558.xlsx']

stop_before_file = ['C:/intern/data/docs_20220525/插值前停靠站点0-90.xlsx', 
                    'C:/intern/data/docs_20220525/插值前停靠站点90-149.xlsx', 
                    'C:/intern/data/docs_20220525/插值前停靠站点150-221.xlsx',
                    'C:/intern/data/docs_20220525/插值前停靠站点244-299.xlsx', 
                    'C:/intern/data/docs_20220525/插值前停靠站点300-399.xlsx',
                    'C:/intern/data/docs_20220525/插值前停靠站点400-499.xlsx',
                    'C:/intern/data/docs_20220525/插值前停靠站点500-558.xlsx']

stop_after_file = ['C:/intern/data/docs_20220525/插值后停靠站点0-90.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点90-149.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点150-221.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点244-299.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点300-399.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点400-499.xlsx',
                   'C:/intern/data/docs_20220525/插值后停靠站点500-558.xlsx']

pass_before_file = ['C:/intern/data/docs_20220525/插值前经过站点0-90.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点90-149.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点150-221.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点244-299.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点300-399.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点400-499.xlsx',
                    'C:/intern/data/docs_20220525/插值前经过站点500-558.xlsx']

pass_after_file = ['C:/intern/data/docs_20220525/插值后经过站点0-90.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点90-149.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点150-221.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点244-299.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点300-399.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点400-499.xlsx',
                   'C:/intern/data/docs_20220525/插值后经过站点500-558.xlsx']

In [134]:
stop_before_df = append_data(stop_before_file)
stop_after_df = append_data(stop_after_file)
pass_before_df = append_data(pass_before_file)
pass_after_df = append_data(pass_after_file)

In [135]:
data = pd.DataFrame(columns=['车次','日期','before_停靠站点数','after_停靠站点数','before_经过站点数','after_经过站点数'])
for f in file:
    each_data = pd.read_excel(f)
    data = data.append(each_data)
data['停靠差别'] = data['after_停靠站点数'] - data['before_停靠站点数']
data['经过差别'] = data['after_经过站点数'] - data['before_经过站点数']

In [162]:
def effect_info(standard_num):
    stopdiff_checi = data[data['停靠差别']>standard_num]['车次'].values
    stopdiff_riqi = data[data['停靠差别']>standard_num]['日期'].values
    stop_df = pd.DataFrame({'车次':stopdiff_checi, '日期':stopdiff_riqi})

    passdiff_checi = data[data['经过差别']>standard_num]['车次'].values
    passdiff_riqi = data[data['经过差别']>standard_num]['日期'].values
    pass_df = pd.DataFrame({'车次':passdiff_checi, '日期':passdiff_riqi})
    stop_df['新增识别站点'] = stop_df.apply(lambda row: check_different_zhandian(stop_before_df,stop_after_df, row['日期'], row['车次']), axis=1)
    pass_df['新增识别站点'] = pass_df.apply(lambda row: check_different_zhandian(pass_before_df,pass_after_df, row['日期'], row['车次']), axis=1)
    stop_df['路线'] = stop_df['车次'].apply(checi_duiying_luxian)
    pass_df['路线'] = pass_df['车次'].apply(checi_duiying_luxian)
    stop_df['属于该车次的新增识别'] = stop_df.apply(lambda row: check_zhandian_included_checi(row['新增识别站点'], row['车次']), axis=1)
    pass_df['属于该车次的新增识别'] = pass_df.apply(lambda row: check_zhandian_included_checi(row['新增识别站点'], row['车次']), axis=1)
    stop_df['新增包含有效站点'] = stop_df['属于该车次的新增识别'].apply(check_xinzeng_valid)
    pass_df['新增包含有效站点'] = pass_df['属于该车次的新增识别'].apply(check_xinzeng_valid)
    stop_df['新增识别站点_距离km'] = stop_df.apply(lambda row: check_zhandian_distance(stop_after_df, row['新增识别站点'], row['日期'], row['车次']), axis=1)
    pass_df['新增识别站点_距离km'] = pass_df.apply(lambda row: check_zhandian_distance(pass_after_df, row['新增识别站点'], row['日期'], row['车次']), axis=1)
    stop_df = stop_df[['车次','路线','日期', '新增识别站点', '新增识别站点_距离km', '属于该车次的新增识别', '新增包含有效站点']]
    pass_df = pass_df[['车次','路线','日期', '新增识别站点', '新增识别站点_距离km', '属于该车次的新增识别', '新增包含有效站点']]
    stop_valid = stop_df[stop_df['新增包含有效站点']]
    pass_valid =pass_df[pass_df['新增包含有效站点']]
    
    return stop_df, stop_valid, pass_df, pass_valid

In [164]:
stop_df, stop_valid, pass_df, pass_valid = effect_info(3)

In [165]:
stop_df.to_excel('C:/intern/data/docs_20220525/新增停靠信息_3.xlsx',index=False)
pass_df.to_excel('C:/intern/data/docs_20220525/新增经过信息_3.xlsx',index=False)
stop_valid.to_excel('C:/intern/data/docs_20220525/有效新增停靠信息_3.xlsx',index=False)
pass_valid.to_excel('C:/intern/data/docs_20220525/有效新增经过信息_3.xlsx',index=False)

In [175]:
#新增有效经过站点识别  路线信息
pass_valid['路线'].value_counts()

Y3     1
Y20    1
Name: 路线, dtype: int64

In [66]:
stop_luxian_df = pd.DataFrame(columns=['路线','车次'])
pass_luxian_df = pd.DataFrame(columns=['路线','车次'])
for each in stopdiff_checi:
    luxian = checi_duiying_luxian(each, shike)
    stop_luxian_df.loc[len(stop_luxian_df.index)] = [luxian, each]
    
for each in passdiff_checi:
    luxian = checi_duiying_luxian(each, shike)
    pass_luxian_df.loc[len(pass_luxian_df.index)] = [luxian, each]

In [68]:
stop_luxian_df

,路线,车次
0,Y8,123
1,Y10,133
2,Y10,133
3,Y10,133
4,Y11,135
5,Y11,144
6,Y11,144
7,Y11,144
8,Y11,145
9,Y11,145
